In [371]:
import pandas as pd
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

import time
import os

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [319]:
# don't show chrome browser
options = Options()
options.add_argument("--headless")

In [320]:
if  getattr(sys, 'frozen', False): 
    chromedriver_path = os.path.join(sys._MEIPASS, "chromedriver.exe")
    driver = webdriver.Chrome(chromedriver_path, options=options)
else:
    driver = webdriver.Chrome(options=options)

################### BELOW IS FOR DESTINATION: DUBAI, DAMMAM, RIYADH #############################
# open ONE shipper's website
driver.get("https://ecomm.one-line.com/one-ecom")
time.sleep(3)

In [321]:
# close initial pop-up
driver.find_element(By.CLASS_NAME,"PrePromotionPopup_close-icon__WOnwe").click()

In [322]:
# close cookie policy pop-up at the bottom
driver.find_element(By.XPATH,"//button[@class='CookiePolicy_closeButton__6sDK1']").click()

In [323]:
# type in origin (input-based dropdown)
origin = driver.find_element(By.ID,"downshift-1-input")
origin.send_keys("PUSAN")
time.sleep(1)
origin.send_keys(Keys.DOWN)
origin.send_keys(Keys.RETURN)

In [324]:
# type in destination (input-based dropdown) Dubai = Jebel Ali
destination = driver.find_element(By.ID,"downshift-2-input")
destination.send_keys("JEBEL ALI")
time.sleep(1)
destination.send_keys(Keys.DOWN)
destination.send_keys(Keys.RETURN)

In [325]:
# next (how many weeks to search for)
driver.find_element(By.XPATH,"//div[@class='Input_inputContainer__TLu7L']/input[@data-cy='port-next-value']").click()

In [326]:
# select next 8 weeks
driver.find_element(By.ID,"next-week-value-56").click()

In [327]:
# click 'Search'
driver.find_element(By.ID,"schedule-box-search-btn-id").click()
time.sleep(2)

In [328]:
# get departure dates
depart_dates_get = driver.find_elements(By.XPATH,"//div[@class='ScheduleItem_date__WLAPW'][@data-cy='new-schedule-departure']")

depart_dates = []
for date in depart_dates_get:
    depart_dates.append(date.text)

In [329]:
# get arrival dates
arrival_dates_get = driver.find_elements(By.XPATH,"//div[@class='ScheduleItem_date__WLAPW'][@data-cy='new-schedule-arrival']")

arrival_dates = []
for date in arrival_dates_get:
    arrival_dates.append(date.text)

In [330]:
# get vessel names
vessel_names_get = driver.find_elements(By.XPATH,"//a[@data-cy='new-schedule-vessel-name']")

vessel_names = []
for vessel in vessel_names_get:
    vessel_names.append(' '.join(vessel.text.split()[:-1]))

In [331]:
# get service lane info
service_lanes_get = driver.find_elements(By.XPATH,"//a[@data-cy='new-schedule-service-land-name']")

service_lanes = []
for lane in service_lanes_get:
    service_lanes.append(lane.text)

In [332]:
# create dataframe for Dubai port (based on ONE shipper's website)
df_shipper_dubai = pd.DataFrame({'departure': depart_dates, 'arrival': arrival_dates, 'vessel': vessel_names, 'route': service_lanes})
df_shipper_dubai.head()

,departure,arrival,vessel,route
0,2023-05-29,2023-06-20,UMM SALAL,AG3
1,2023-06-04,2023-06-25,HMM RAON,AG3
2,2023-06-16,2023-07-07,AL JMELIYAH,AG3
3,2023-06-18,2023-07-09,BERLIN EXPRESS,AG3
4,2023-06-25,2023-07-16,YM WELLHEAD,AG3


In [333]:
# open Busan port terminal website
driver.get("https://www.hpnt.co.kr/infoservice/vessel/vslScheduleList.jsp")

In [334]:
from datetime import date
# get dates for two months from today
today = date.today()
after_2months = str(today + relativedelta(months=+2))

In [335]:
# change ending date for search
end_date = driver.find_element(By.ID,"strdEdDate")
end_date.clear()
end_date.send_keys(after_2months)

In [336]:
# type in route and click 'search'
route = driver.find_element(By.ID,"route")
route.send_keys("AG3")
driver.find_element(By.ID,"submitbtn").click()

In [337]:
# get vessel's name
vessel_terminal_get = driver.find_elements(By.XPATH,"//tr[@class='color_planned']/td[5]")
vessel_terminal = []
for vessel in vessel_terminal_get:
    vessel_terminal.append(vessel.text)

In [338]:
# get expected anchoring date
anchor_get = driver.find_elements(By.XPATH,"//tr[@class='color_planned']/td[8]")
anchor_dates = []
for date in anchor_get:
    anchor_dates.append(date.text)

In [339]:
# get expected departure date
depart_terminal_get = driver.find_elements(By.XPATH,"//tr[@class='color_planned']/td[9]")
depart_dates_terminal = []
for date in depart_terminal_get:
    depart_dates_terminal.append(date.text)

In [340]:
# create dataframe for Dubai port (based on Busan port terminal's website)
df_terminal_dubai = pd.DataFrame({'vessel': vessel_terminal, 'anchor': anchor_dates, 'departure_terminal': depart_dates_terminal})
df_terminal_dubai

,vessel,anchor,departure_terminal
0,HMM GAON,2023-05-19 17:00,2023-05-20 11:00
1,UMM SALAL,2023-05-26 14:00,2023-05-28 04:00
2,TAYMA EXPRESS,2023-05-31 09:00,2023-06-01 16:00
3,HMM RAON,2023-06-02 17:00,2023-06-04 07:00
4,AL JMELIYAH,2023-06-09 17:00,2023-06-11 07:00


In [341]:
# merge ONE shipper's and Busan terminal's info into one DataFrame
df_dubai = df_shipper_dubai.merge(df_terminal_dubai, on='vessel', how='left')
df_dubai

,departure,arrival,vessel,route,anchor,departure_terminal
0,2023-05-29,2023-06-20,UMM SALAL,AG3,2023-05-26 14:00,2023-05-28 04:00
1,2023-06-04,2023-06-25,HMM RAON,AG3,2023-06-02 17:00,2023-06-04 07:00
2,2023-06-16,2023-07-07,AL JMELIYAH,AG3,2023-06-09 17:00,2023-06-11 07:00
3,2023-06-18,2023-07-09,BERLIN EXPRESS,AG3,NaN,NaN
4,2023-06-25,2023-07-16,YM WELLHEAD,AG3,NaN,NaN
5,2023-07-02,2023-07-23,UMM QARN,AG3,NaN,NaN
6,2023-07-09,2023-07-30,HMM HANUL,AG3,NaN,NaN


In [342]:
################### BELOW IS FOR DESTINATION: JEDDAH ############################
# open ONE shipper's website
driver.get("https://ecomm.one-line.com/one-ecom")
time.sleep(3)

In [343]:
# type in origin (input-based dropdown)
origin = driver.find_element(By.ID,"downshift-1-input")
origin.send_keys("PUSAN")
time.sleep(1)
origin.send_keys(Keys.DOWN)
origin.send_keys(Keys.RETURN)

In [344]:
# type in destination (input-based dropdown)
destination = driver.find_element(By.ID,"downshift-2-input")
destination.send_keys("JEDDAH")
time.sleep(1)
destination.send_keys(Keys.DOWN)
destination.send_keys(Keys.RETURN)

In [345]:
# next (how many weeks to search for)
driver.find_element(By.XPATH,"//div[@class='Input_inputContainer__TLu7L']/input[@data-cy='port-next-value']").click()

In [346]:
# select next 8 weeks
driver.find_element(By.ID,"next-week-value-56").click()

In [347]:
# click 'Search'
driver.find_element(By.ID,"schedule-box-search-btn-id").click()
time.sleep(2)

In [348]:
# get departure dates
depart_dates_get = driver.find_elements(By.XPATH,"//div[@class='ScheduleItem_date__WLAPW'][@data-cy='new-schedule-departure']")

depart_dates = []
for date in depart_dates_get:
    depart_dates.append(date.text)

In [349]:
# get arrival dates
arrival_dates_get = driver.find_elements(By.XPATH,"//div[@class='ScheduleItem_date__WLAPW'][@data-cy='new-schedule-arrival']")

arrival_dates = []
for date in arrival_dates_get:
    arrival_dates.append(date.text)

In [350]:
# get vessel's names
vessel_names_get = driver.find_elements(By.XPATH,"//a[@data-cy='new-schedule-vessel-name']")

vessel_names = []
for vessel in vessel_names_get:
    vessel_names.append(' '.join(vessel.text.split()[:-1]))

In [351]:
# get service lane info
service_lanes_get = driver.find_elements(By.XPATH,"//a[@data-cy='new-schedule-service-land-name']")

service_lanes = []
for lane in service_lanes_get:
    service_lanes.append(lane.text)

In [352]:
# create dataframe for Jeddah port (based on ONE shipper's website)
df_shipper_jeddah = pd.DataFrame({'departure': depart_dates, 'arrival': arrival_dates, 'vessel': vessel_names, 'route': service_lanes})
df_shipper_jeddah

,departure,arrival,vessel,route
0,2023-05-24,2023-06-17,YM MILESTONE,AR1
1,2023-05-26,2023-06-20,ANTWERPEN EXPRESS,MD3
2,2023-05-31,2023-06-24,WAN HAI 613,AR1
3,2023-06-09,2023-07-04,YM WONDERLAND,MD3
4,2023-06-10,2023-07-01,YM MODESTY,AR1
5,2023-06-14,2023-07-08,YM PLUM,AR1
6,2023-06-16,2023-07-11,PARIS EXPRESS,MD3
7,2023-06-21,2023-07-15,YM COSMOS,AR1
8,2023-06-23,2023-07-18,LEVERKUSEN EXPRESS,MD3
9,2023-07-01,2023-07-25,YM WINDOW,MD3


In [353]:
# open Busan port terminal website
driver.get("http://www.hjnc.co.kr/esvc/vessel/berthScheduleT")

In [354]:
# click 'manual input'
driver.find_element(By.XPATH,"//input[@name='chkPeriod'][@value='mm']").click()

In [355]:
# change ending date to two months laterfor search
month = driver.find_element(By.XPATH,"//select[@id='selEndMonth']")
month.send_keys(Keys.DOWN)
month.send_keys(Keys.DOWN)

In [356]:
# search for 'MD3' service lane (Jeddah has two lanes: MD3 and AR1)
driver.find_element(By.ID,"searchRoute").send_keys("MD3")
driver.find_element(By.ID,"btnSearch").click()
time.sleep(2)

In [357]:
# get vessel names, anchor dates, and departure dates
md3_get_vessel = driver.find_elements(By.XPATH,"//tbody/tr[@role='row']/td[5]")
md3_get_anchor = driver.find_elements(By.XPATH,"//tbody/tr[@role='row']/td[9]")
md3_get_depart = driver.find_elements(By.XPATH,"//tbody/tr[@role='row']/td[10]")

vessel_names = []
anchor_dates = []
depart_dates = []

for vessel in md3_get_vessel:
    vessel_names.append(vessel.text)
    
for date in md3_get_anchor:
    anchor_dates.append(date.text)
    
for date in md3_get_depart:
    depart_dates.append(date.text)

In [358]:
# create dataframe for Jeddah port, 'MD3' lane (based on Busan port terminal's website)
df_terminal_md3 = pd.DataFrame({'vessel': vessel_names, 'anchor': anchor_dates, 'departure_terminal': depart_dates})
df_terminal_md3

,vessel,anchor,departure_terminal
0,YM WORLD,2023-05-18 13:05,2023-05-19 17:00
1,ANTWERPEN EXPRESS,2023-05-24 22:00,2023-05-25 10:00
2,YM WELLSPRING,2023-05-26 17:00,2023-05-27 21:00
3,YM WONDERLAND,2023-06-07 17:00,2023-06-08 21:00
4,PARIS EXPRESS,2023-06-14 17:00,2023-06-15 21:00
5,LEVERKUSEN EXPRESS,2023-06-21 17:00,2023-06-22 21:00
6,YM WINDOW,2023-06-29 17:00,2023-06-30 21:00


In [359]:
# now search for 'AR1' lane 
driver.find_element(By.ID,"searchRoute").clear()
driver.find_element(By.ID,"searchRoute").send_keys("AR1")
driver.find_element(By.ID,"btnSearch").click()

In [360]:
# get vessel names, anchor dates, and departure dates
ar1_get_vessel = driver.find_elements(By.XPATH,"//tbody/tr[@role='row']/td[5]")
ar1_get_anchor = driver.find_elements(By.XPATH,"//tbody/tr[@role='row']/td[9]")
ar1_get_depart = driver.find_elements(By.XPATH,"//tbody/tr[@role='row']/td[10]")

vessel_names = []
anchor_dates = []
depart_dates = []

for vessel in ar1_get_vessel:
    vessel_names.append(vessel.text)
    
for date in ar1_get_anchor:
    anchor_dates.append(date.text)
    
for date in ar1_get_depart:
    depart_dates.append(date.text)

In [361]:
# create dataframe for Jeddah port, 'AR1' lane (based on Busan port terminal's website)
df_terminal_ar1 = pd.DataFrame({'vessel': vessel_names, 'anchor': anchor_dates, 'departure_terminal': depart_dates})
df_terminal_ar1

,vessel,anchor,departure_terminal
0,YM MILESTONE,2023-05-24 00:00,2023-05-24 16:00
1,WAN HAI 613,2023-05-31 00:00,2023-05-31 16:00
2,YM MODESTY,2023-06-07 00:00,2023-06-07 14:00
3,YM PLUM,2023-06-13 23:00,2023-06-14 15:00
4,YM COSMOS,2023-06-21 00:00,2023-06-21 14:00


In [362]:
# concatenate 'MD3' and 'AR1' DataFrames
df_concat = pd.concat([df_terminal_md3, df_terminal_ar1])

In [363]:
# merge ONE shipper's and Busan terminal's info into one DataFrame
df_jeddah = df_shipper_jeddah.merge(df_concat, on='vessel', how='left')

In [364]:
# change 'anchor' column into datetime
df_dubai['anchor'] = pd.to_datetime(df_dubai['anchor'])
df_jeddah['anchor'] = pd.to_datetime(df_jeddah['anchor'])

In [365]:
# containers can enter CY three days before the anchor date, so create new column for it
df_dubai['CY_entry'] = df_dubai['anchor'].dt.date - timedelta(days=3)
df_jeddah['CY_entry'] = df_jeddah['anchor'].dt.date - timedelta(days=3)

In [366]:
# convert DataFrames to excel
writer =  pd.ExcelWriter('ship_schedule.xlsx')
df_dubai.to_excel(writer, sheet_name='dubai', index=False)
df_jeddah.to_excel(writer, sheet_name='jeddah', index=False)

In [367]:
# set column length and alignment for better readability
for column in df_dubai:
    column_length = max(df_dubai[column].astype(str).map(len).max(), len(str(column)))
    col_idx = df_dubai.columns.get_loc(column)
    writer.sheets['dubai'].set_column(col_idx, col_idx, column_length+3)
    
for column in df_jeddah:
    column_length = max(df_jeddah[column].astype(str).map(len).max(), len(str(column)))
    col_idx = df_jeddah.columns.get_loc(column)
    writer.sheets['jeddah'].set_column(col_idx, col_idx, column_length+3)

workbook = writer.book
worksheet1 = writer.sheets["dubai"]
worksheet2 = writer.sheets["jeddah"]

fmt = workbook.add_format()
fmt.set_align('right')

worksheet1.set_column('F:F', 22, fmt)
worksheet2.set_column('F:F', 22, fmt)
    
writer.save()
writer.close()

C:\Users\USER\anaconda3\lib\site-packages\xlsxwriter\workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


In [368]:
# open excel file
os.system("start EXCEL.EXE ship_schedule.xlsx");